In [1]:
import sys
sys.path.append('../src')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np
import loading_json as load

In [2]:
T = 50
road_1 = rd.Road(2, 50, 5, torch.tensor([50.0], requires_grad=True), [],
               initial=lambda x : torch.ones_like(x) * 0.2,
               left_pos=(-1, 2), right_pos=(7,2),
               periodic=True, id = "road_1_fw")

road_2 = rd.Road(2, 50, 5, torch.tensor([50.0], requires_grad=True), [],
               initial=lambda x : torch.ones_like(x) * 0.2,
               left_pos=(-1, 4), right_pos=(7,4),
               periodic=True, id = "road_2_fw", max_dens=4)

road_3 = rd.Road(2, 50, 5, torch.tensor([50.0], requires_grad=True), [],
               initial=lambda x : torch.ones_like(x) * 0.2,
               left_pos=(-1, 6.2), right_pos=(7,6.2),
               periodic=True, id = "road_3_fw")

road_4 = rd.Road(2, 50, 5, torch.tensor([50.0], requires_grad=True), [],
               initial=lambda x : torch.ones_like(x) * 0.2,
               left_pos=(7,6.2), right_pos=(-1, 6.2),
               periodic=True, id = "road_3_bw")

road_5 = rd.Road(2, 50, 5, torch.tensor([50.0], requires_grad=True), [],
               initial=lambda x : torch.ones_like(x) * 0.2,
               left_pos=(-1, 8.2), right_pos=(7,8.2),
               periodic=True, id = "road_4_fw")

road_6 = rd.Road(2, 50, 5, torch.tensor([50.0], requires_grad=True), [],
               initial=lambda x : torch.ones_like(x) * 0.6,
               left_pos=(7,8.2), right_pos=(-1, 8.2),
               periodic=True, id = "road_4_bw")


roads = [road_1, road_2, road_3, road_4, road_5, road_6]
network = nw.RoadNetwork(roads, [], T)

In [3]:
ids = ["road_1_fw"]
stops = [("road_1_fw", 40)]
times = [80]
bus_1 = bus.Bus(ids, stops, times, network)

ids = ["road_2_fw"]
stops = [("road_2_fw", 40)]
times = [80]
bus_2 = bus.Bus(ids, stops, times, network)

ids = ["road_3_fw"]
stops = [("road_3_fw", 40)]
times = [80]
bus_3 = bus.Bus(ids, stops, times, network)

ids = ["road_4_fw"]
stops = [("road_4_fw", 40)]
times = [80]
bus_4 = bus.Bus(ids, stops, times, network)

bus_network = nw.RoadNetwork(roads, [], T, busses=[bus_1, bus_2, bus_3, bus_4])

In [4]:
densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

Bus  reached bus stop 0 at time 3.899634838104248, should wait for 75.97452545166016 seconds
Bus  reached bus stop 0 at time 3.899634838104248, should wait for 75.9752197265625 seconds
Bus  reached bus stop 0 at time 3.899634838104248, should wait for 75.97509002685547 seconds
Bus  reached bus stop 0 at time 3.899634838104248, should wait for 75.97505187988281 seconds


In [5]:
# Checking the slowdown factors:
dt = 0.5
t = torch.tensor(50.0)
slowdown_factors = [torch.ones(road.N_internal+1) for road in bus_network.roads]
slowdown_indexes = []
for bus in bus_network.busses:
    slowdown_factors, slowing_idx = bus_network.update_position_of_bus(bus, dt, t, slowdown_factors)
    if slowing_idx is not None:
        slowdown_indexes.append(slowing_idx)

In [6]:
for idx in slowdown_indexes:
    print(slowdown_factors[idx])
    # print(idx)

tensor([1.0000, 1.0000, 1.0000, 0.3384, 0.4062, 0.9792, 0.9998, 1.0000, 1.0000,
        1.0000, 1.0000], grad_fn=<CopySlices>)
tensor([1.0000, 1.0000, 1.0000, 0.9271, 0.9372, 0.9979, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000], grad_fn=<CopySlices>)
tensor([1.0000, 1.0000, 1.0000, 0.8181, 0.8367, 0.9943, 0.9999, 1.0000, 1.0000,
        1.0000, 1.0000], grad_fn=<CopySlices>)
tensor([1.0000, 1.0000, 1.0000, 0.7850, 0.8070, 0.9933, 0.9999, 1.0000, 1.0000,
        1.0000, 1.0000], grad_fn=<CopySlices>)


In [7]:
densities = load.convert_from_tensor(densities)
queues = load.convert_from_tensor(queues)
bus_lengths = load.convert_from_tensor(bus_lengths)

In [8]:
import opengl_generate_gif as opengl

In [9]:
# Create gif:
opengl.draw_busses_w_densities(bus_network, [bus_1, bus_2, bus_3, bus_4], bus_lengths, densities, "bus_slowdown_example.gif", "white_background.png")

End of simulation reached!
Saving GIF as: bus_slowdown_example.gif


: 